# Modeling: GBM & Healthy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split


Import preprocessed files

In [ ]:
metadata_filepath = "data/processed/metadata_gbm_healthy.csv"
expression_filepath = "data/processed/expression_gbm_healthy.csv"

In [ ]:
gene_expression_df = pd.read_csv(expression_filepath, index_col='sample_id')


In [ ]:
metadata_df = pd.read_csv(
    metadata_filepath, 
    index_col='sample_id'
)
metadata_df

## Explorative Data Analysis

In [ ]:
X = gene_expression_df
y = metadata_df

print(f"Loaded X (features) with shape: {X.shape}")
print(f"Loaded y (labels) with shape: {y.shape}")

# We already scaled the data, so we can run PCA directly
pca = PCA(n_components=2)
data_pca = pca.fit_transform(X)

# Get the variance explained by each component
variance_explained = pca.explained_variance_ratio_
pc1_var = variance_explained[0] * 100
pc2_var = variance_explained[1] * 100

print("PCA complete.")

# Put the PCA results into a DataFrame
pca_df = pd.DataFrame(
    data_pca,
    columns=['PC1', 'PC2'],
    index=X.index
)

# Add the 1, 2, 3 labels from our 'y' file
pca_df['label_id'] = y['label']

# Create a meaningful text label for plotting
text_label_map = {1: 'Healthy', 3: 'GBM'}
pca_df['Diagnosis'] = pca_df['label_id'].map(text_label_map)

# Infer the batch from the sample_id (TCGA or GTEX)
pca_df['Batch (Study)'] = ['TCGA' if 'TCGA' in idx else 'GTEX' for idx in pca_df.index]

print("PCA results merged with labels. Ready to plot.")
display(pca_df.head())

In [ ]:
plt.figure(figsize=(10, 7))
sns.scatterplot(
    data=pca_df,
    x='PC1',
    y='PC2',
    hue='Diagnosis',
    palette={'Healthy': '#457B9D', 'GBM': '#E63946'},
    alpha=0.7,

)
plt.tight_layout()
plt.title('PCA - Binary Classification: Healthy vs GBM', fontsize=15),
plt.xlabel(f'PC1 ({pc1_var:.1f}% variance)', fontsize=12)
plt.ylabel(f'PC2 ({pc2_var:.1f}% variance)', fontsize=12)
plt.show()


### Top 30 most impactful genes

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import f_classif

# We need y as a 1D array for the test
y_ravel = y.values.ravel()

print(f"Loaded X ({X.shape}) and y ({y_ravel.shape})")

gene_variances = X.var(axis=0)

# We will only keep genes with a variance > 0.01
# This is a safe threshold to remove "dead" genes (which have 0 variance)
# and "noisy" genes (which have near-0 variance)
variance_filter = gene_variances > 0.01 

X_active_genes = X.loc[:, variance_filter]

genes_removed = X.shape[1] - X_active_genes.shape[1]
print(f"Removed {genes_removed} constant or low-variance genes.")
print(f"Running test on remaining {X_active_genes.shape[1]} active genes.")

f_scores, p_values = f_classif(X_active_genes, y_ravel)
print("ANOVA F-test complete.")

gene_impact_df = pd.DataFrame({
    'gene_id': X_active_genes.columns,
    'f_score': f_scores,
    'p_value': p_values
})

gene_impact_df = gene_impact_df.dropna()

# Sort by the F-score (highest first)
top_30_genes = gene_impact_df.sort_values(by='f_score', ascending=False).head(30)

print("\n--- Top 30 Most Impactful Genes (Corrected) ---")
display(top_30_genes)

plt.figure(figsize=(10, 12))
sns.barplot(
    x='f_score',
    y='gene_id',
    data=top_30_genes,
    palette='viridis',
    hue='gene_id',
    legend=False
)
plt.title('Top 30 Most Impactful Genes (ANOVA F-score)', fontsize=16)
plt.xlabel('Impact Score (F-score)', fontsize=12)
plt.ylabel('Gene ID', fontsize=12)
plt.show()

#### Plot top gene

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

top_gene_id = top_30_genes.iloc[0]['gene_id']

print(f"Plotting expression for top gene: {top_gene_id}")

plot_df = pd.DataFrame(X[top_gene_id])
plot_df['label'] = y['label']
text_label_map = {1: 'Healthy', 3: 'GBM'}
plot_df['Diagnosis'] = plot_df['label'].map(text_label_map)

plt.figure(figsize=(10, 7))

sns.boxplot(
    data=plot_df,
    x='Diagnosis',
    y=top_gene_id,
    palette={'Healthy': '#457B9D', 'GBM': '#E63946'},
    order=['Healthy', 'GBM'], 
    hue='Diagnosis', 
    legend=False     
)

sns.stripplot(
    data=plot_df,
    x='Diagnosis',
    y=top_gene_id,
    color='black',
    alpha=0.2, 
    jitter=0.2, 
    order=['Healthy', 'GBM']
)

plt.title(f'Expression of Top Gene: {top_gene_id}', fontsize=16)
plt.xlabel('Diagnosis', fontsize=12)
plt.ylabel('Scaled Gene Expression', fontsize=12)
plt.show()

## Data Driven feature selection

### LASSO

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)
y_train_ravel = y_train.values.ravel()
print(f"Split data into {len(X_train)} training and {len(X_test)} test samples.")

print("Running Data-Driven Selection (Lasso)...")

base_lasso_estimator = LogisticRegression(
    penalty='l1',
    solver='liblinear',
    class_weight='balanced',
    C=0.1,  
    random_state=42
)

ovr_classifier = OneVsRestClassifier(base_lasso_estimator)

ovr_classifier.fit(X_train, y_train_ravel)


all_coefs = np.vstack([est.coef_ for est in ovr_classifier.estimators_])


features_selected_mask = np.sum(np.abs(all_coefs), axis=0) > 1e-5

feature_list_A = X_train.columns[features_selected_mask]

print(f"Lasso (C=0.1) selected {len(feature_list_A)} genes.")
print("\n--- Top Genes Selected by Lasso ---")
print(list(feature_list_A[:20])) # Print first 20

# --- 5. Plot a Boxplot for the FIRST gene from THIS list ---
if len(feature_list_A) > 0:
    top_gene_id_lasso = feature_list_A[0]
    print(f"\nPlotting expression for top Lasso gene: {top_gene_id_lasso}")

    # Create a DataFrame for Plotting (using the full X and y)
    plot_df = pd.DataFrame(X[top_gene_id_lasso])
    plot_df['label'] = y['label']
    text_label_map = {1: 'Healthy', 3: 'GBM'}
    plot_df['Diagnosis'] = plot_df['label'].map(text_label_map)

    # Create the Boxplot
    plt.figure(figsize=(10, 7))
    sns.boxplot(
        data=plot_df,
        x='Diagnosis',
        y=top_gene_id_lasso,
        palette={'Healthy': '#457B9D', 'GBM': '#E63946'},
        order=['Healthy', 'GBM'],
        hue='Diagnosis',
        legend=False
    )
    sns.stripplot(
        data=plot_df,
        x='Diagnosis',
        y=top_gene_id_lasso,
        color='black',
        alpha=0.2,
        jitter=0.2,
        order=['Healthy', 'GBM']
    )
    plt.title(f'Expression of Top Lasso Gene: {top_gene_id_lasso}', fontsize=16)
    plt.xlabel('Diagnosis', fontsize=12)
    plt.ylabel('Scaled Gene Expression', fontsize=12)
    plt.show()

else:
    print("Lasso selected 0 genes. Your C value (0.1) might be too strong.")
    print("Try increasing C (e.g., C=0.5 or C=1.0) to select more features.")

### Recursive Feature Elimination

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns

print("\nRunning Data-Driven Selection (RFE)...")

rfe_estimator = LogisticRegression(
    solver='lbfgs',  
    class_weight='balanced',
    random_state=42
)

selector_rfe = RFE(
    estimator=rfe_estimator,
    n_features_to_select=250,
    step=0.1 
)

print("Fitting RFE... (This may take a moment)")
selector_rfe.fit(X_train, y_train_ravel)

features_selected_mask_rfe = selector_rfe.get_support()
feature_list_B = X_train.columns[features_selected_mask_rfe]

print(f"RFE selected {len(feature_list_B)} genes.")
print("\n--- First 20 Genes Selected by RFE ---")
print(list(feature_list_B[:20]))

if len(feature_list_B) > 0:
    top_gene_id_rfe = feature_list_B[0]
    print(f"\nPlotting expression for top RFE gene: {top_gene_id_rfe}")

    plot_df = pd.DataFrame(X[top_gene_id_rfe])
    plot_df['label'] = y['label']
    text_label_map = {1: 'Healthy', 3: 'GBM'}
    plot_df['Diagnosis'] = plot_df['label'].map(text_label_map)

    # Create the Boxplot
    plt.figure(figsize=(10, 7))
    sns.boxplot(
        data=plot_df,
        x='Diagnosis',
        y=top_gene_id_rfe,
        palette={'Healthy': '#457B9D', 'GBM': '#E63946'},
        order=['Healthy', 'GBM'],
        hue='Diagnosis',
        legend=False
    )
    sns.stripplot(
        data=plot_df,
        x='Diagnosis',
        y=top_gene_id_rfe,
        color='black',
        alpha=0.2,
        jitter=0.2,
        order=['Healthy', 'GBM']
    )
    plt.title(f'Expression of Top RFE Gene: {top_gene_id_rfe}', fontsize=16)
    plt.xlabel('Diagnosis', fontsize=12)
    plt.ylabel('Scaled Gene Expression', fontsize=12)
    plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay

# We use the gene list from our Lasso (feature_list_A)
X_train_lasso = X_train[feature_list_A]
X_test_lasso = X_test[feature_list_A]

print(f"--- Training Model 1: Lasso Features ({len(feature_list_A)} genes) ---")
print(f"Training data shape: {X_train_lasso.shape}")

# We use the same Random Forest model as before
rand_forest_lasso = RandomForestClassifier(
    n_estimators=100,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rand_forest_lasso.fit(X_train_lasso, y_train_ravel)
print("Model training complete.")

y_pred_lasso = rand_forest_lasso.predict(X_test_lasso)

print("\n--- Results for Lasso Feature Model ---")
acc_lasso = accuracy_score(y_test, y_pred_lasso)
print(f"Accuracy: {acc_lasso * 100:.2f}%")

target_names = ['Healthy', 'GBM']
print(classification_report(y_test, y_pred_lasso, target_names=target_names, zero_division=0))

print("\nPlotting Confusion Matrix for Lasso Model:")
fig, ax = plt.subplots(figsize=(8, 6))
ConfusionMatrixDisplay.from_predictions(
    y_test,
    y_pred_lasso,
    display_labels=target_names,
    cmap='Blues',
    ax=ax
)
ax.set_title('Lasso Feature Model Confusion Matrix')
plt.show()

### LASSO with Under Sampling

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay
from imblearn.under_sampling import RandomUnderSampler 

# This assumes X_train, X_test, y_train, y_test, and feature_list_A
# (with 2253 genes) are all in memory.
X_train_lasso = X_train[feature_list_A]
X_test_lasso = X_test[feature_list_A]

print(f"--- Training Model 1: Lasso Features ({len(feature_list_A)} genes) ---")
print(f"Original training shape: {X_train_lasso.shape}")
print(f"Original label counts:\n{y_train['label'].value_counts()}")

# We create an undersampler. It will match the size of all classes
# to the size of the *smallest* class (GBM).
rus = RandomUnderSampler(random_state=42)
X_train_balanced, y_train_balanced_ravel = rus.fit_resample(X_train_lasso, y_train_ravel)

print("\n--- After Undersampling ---")
print(f"Balanced training shape: {X_train_balanced.shape}")
# This will show a small, but perfectly balanced, set of labels
print(f"Balanced label counts:\n{pd.Series(y_train_balanced_ravel).value_counts()}")

# We can now *remove* class_weight='balanced' because the data is already balanced.
rand_forest_lasso = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

# We fit on the NEW balanced data
rand_forest_lasso.fit(X_train_balanced, y_train_balanced_ravel)
print("\nModel training complete.")

# We still test on the *original, imbalanced* test set
y_pred_lasso = rand_forest_lasso.predict(X_test_lasso)

print("\n--- Results for Lasso Feature Model (Undersampled) ---")
acc_lasso = accuracy_score(y_test, y_pred_lasso)
print(f"Accuracy: {acc_lasso * 100:.2f}%")

target_names = ['Healthy', 'GBM']
print(classification_report(y_test, y_pred_lasso, target_names=target_names))

print("\nPlotting Confusion Matrix for Lasso Model:")
fig, ax = plt.subplots(figsize=(8, 6))
ConfusionMatrixDisplay.from_predictions(
    y_test,
    y_pred_lasso,
    display_labels=target_names,
    cmap='Blues',
    ax=ax
)
ax.set_title('Lasso Feature Model Confusion Matrix (Undersampled)')
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay
from imblearn.under_sampling import RandomUnderSampler 

# We use the gene list from our RFE (feature_list_B)
X_train_rfe = X_train[feature_list_B]
X_test_rfe = X_test[feature_list_B]

print(f"\n--- Training Model 2: RFE Features ({len(feature_list_B)} genes) ---")
print(f"Original training shape: {X_train_rfe.shape}")

rus = RandomUnderSampler(random_state=42)
X_train_balanced, y_train_balanced_ravel = rus.fit_resample(X_train_rfe, y_train_ravel)

print("\n--- After Undersampling ---")
print(f"Balanced training shape: {X_train_balanced.shape}")
print(f"Balanced label counts:\n{pd.Series(y_train_balanced_ravel).value_counts()}")

rand_forest_rfe = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

# We fit on the NEW balanced data
rand_forest_rfe.fit(X_train_balanced, y_train_balanced_ravel)
print("\nModel training complete.")

y_pred_rfe = rand_forest_rfe.predict(X_test_rfe)

print("\n--- Results for RFE Feature Model (Undersampled) ---")
acc_rfe = accuracy_score(y_test, y_pred_rfe)
print(f"Accuracy: {acc_rfe * 100:.2f}%")

target_names = ['Healthy', 'GBM']
print(classification_report(y_test, y_pred_rfe, target_names=target_names))

print("\nPlotting Confusion Matrix for RFE Model:")
fig, ax = plt.subplots(figsize=(8, 6))
ConfusionMatrixDisplay.from_predictions(
    y_test,
    y_pred_rfe,
    display_labels=target_names,
    cmap='Blues',
    ax=ax
)
ax.set_title('RFE Feature Model Confusion Matrix (Undersampled)')
plt.show()

## Knowledge Driven feature selection

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay
from imblearn.under_sampling import RandomUnderSampler 
import numpy as np

map_filepath = "data/processed/gene_map.csv"
print(f"Loading gene map from: {map_filepath}")

try:
    map_df = pd.read_csv(map_filepath)
    gene_symbol_map = pd.Series(map_df.gene_id_clean.values, index=map_df.gene_name).to_dict()
    gene_symbol_map_reverse = {v: k for k, v in gene_symbol_map.items()}
    print("Gene map loaded successfully.")
except FileNotFoundError:
    print(f"FATAL ERROR: '{map_filepath}' not found.")
    print("Please run the 'create_gene_map.py' script first.")
    raise

knowledge_driven_genes = [
    'IDH1', 'EGFR', 'TERT', 'ATRX', 'PTEN', 'MGMT', 'TP53', 
    'PDGFRA', 'CIC', 'FUBP1', 'CDKN2A', 'PIK3CA'
]

feature_list_C = []
genes_not_found = []

for gene_symbol in knowledge_driven_genes:
    try:
        ensembl_id = gene_symbol_map[gene_symbol]
        
        if ensembl_id in X_train.columns:
            feature_list_C.append(ensembl_id)
        else:
            genes_not_found.append(f"{gene_symbol} (ID: {ensembl_id}, not in final data)")
            
    except KeyError:
        genes_not_found.append(f"{gene_symbol} (Symbol not in map)")

print(f"\n--- Training Model 3: Literature-Driven Features ---")
print(f"Total genes found: {len(feature_list_C)}")
print(f"Genes found: {feature_list_C}")
if genes_not_found:
    print(f"Warning: Could not find these genes: {genes_not_found}")

X_train_lit = X_train[feature_list_C]
X_test_lit = X_test[feature_list_C]

rus = RandomUnderSampler(random_state=42)
X_train_balanced, y_train_balanced_ravel = rus.fit_resample(X_train_lit, y_train_ravel)
print(f"Balanced training shape: {X_train_balanced.shape}")

rand_forest_lit = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

rand_forest_lit.fit(X_train_balanced, y_train_balanced_ravel)
print("Model training complete.")

y_pred_lit = rand_forest_lit.predict(X_test_lit)

print("\n--- Results for Literature-Driven Model (Undersampled) ---")
acc_lit = accuracy_score(y_test, y_pred_lit)
print(f"Accuracy: {acc_lit * 100:.2f}%")

target_names = ['Healthy', 'GBM']
print(classification_report(y_test, y_pred_lit, target_names=target_names))

print("\nPlotting Confusion Matrix for Literature-Driven Model:")
fig, ax = plt.subplots(figsize=(8, 6))
ConfusionMatrixDisplay.from_predictions(
    y_test,
    y_pred_lit,
    display_labels=target_names,
    cmap='Blues',
    ax=ax
)
ax.set_title('Literature-Driven Model Confusion Matrix (Undersampled)')
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay

print("\n--- Training Final Model: Knowledge-Driven + class_weight='balanced' ---")
print(f"Training on {len(feature_list_C)} expert-selected genes.")
print(f"Training data shape: {X_train.shape}")
print(f"Original label counts:\n{y_train['label'].value_counts()}")

X_train_lit = X_train[feature_list_C]
X_test_lit = X_test[feature_list_C]

# We use class_weight='balanced' on the *original, imbalanced* data
rand_forest_final = RandomForestClassifier(
    n_estimators=100,
    class_weight='balanced',  # <-- This is the key
    random_state=42,
    n_jobs=-1
)

# Fit on the original, imbalanced (but feature-selected) training data
rand_forest_final.fit(X_train_lit, y_train_ravel)
print("\nModel training complete.")

y_pred_final = rand_forest_final.predict(X_test_lit)

print("\n--- Results for Final Model ---")
acc_final = accuracy_score(y_test, y_pred_final)
print(f"Accuracy: {acc_final * 100:.2f}%")

target_names = ['Healthy', 'GBM']
print(classification_report(y_test, y_pred_final, target_names=target_names))

# --- 4. Plot Confusion Matrix ---
print("\nPlotting Confusion Matrix for Final Model:")
fig, ax = plt.subplots(figsize=(8, 6))
ConfusionMatrixDisplay.from_predictions(
    y_test,
    y_pred_final,
    display_labels=target_names,
    cmap='Blues',
    ax=ax
)
ax.set_title('Final Model (Knowledge-Driven + Balanced Weights)')
plt.show()

## Advanced Model Analysis

### SHAP Analysis

In [ ]:
import shap
import numpy as np

print("Generating SHAP values for knowledge-driven model")

explainer = shap.TreeExplainer(rand_forest_lit)
shap_values_raw = explainer.shap_values(X_test_lit)

print(f"Raw SHAP values type: {type(shap_values_raw)}")
if isinstance(shap_values_raw, list):
    print(f"List length: {len(shap_values_raw)}")
    for i, sv in enumerate(shap_values_raw):
        print(f"  Class {i} shape: {sv.shape}")
    shap_values_class = shap_values_raw[1]
elif isinstance(shap_values_raw, np.ndarray):
    print(f"Array shape: {shap_values_raw.shape}")
    if len(shap_values_raw.shape) == 3:
        shap_values_class = shap_values_raw[:, :, 1]
    else:
        shap_values_class = shap_values_raw
else:
    shap_values_class = shap_values_raw

print(f"Final SHAP values shape: {shap_values_class.shape}")
print(f"Expected value: {explainer.expected_value}")

In [ ]:
gene_names_display = [gene_symbol_map_reverse.get(col, col) for col in X_test_lit.columns]

plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values_class, X_test_lit, feature_names=gene_names_display, show=False)
plt.title("SHAP Feature Importance for 12 Knowledge-Driven Genes")
plt.tight_layout()
plt.show()

In [ ]:
misclassified_indices = np.where(y_pred_lit != y_test.values.ravel())[0]

if len(misclassified_indices) > 0:
    sample_idx = misclassified_indices[0]
    
    true_label = y_test.values.ravel()[sample_idx]
    pred_label = y_pred_lit[sample_idx]
    
    print(f"\nMisclassified sample analysis:")
    print(f"True label: {true_label}, Predicted: {pred_label}")
    
    base_val = explainer.expected_value[1] if isinstance(explainer.expected_value, (list, np.ndarray)) else explainer.expected_value
    
    shap.waterfall_plot(
        shap.Explanation(
            values=shap_values_class[sample_idx],
            base_values=base_val,
            data=X_test_lit.iloc[sample_idx].values,
            feature_names=gene_names_display
        ),
        show=False
    )
    plt.title(f"SHAP Explanation for Misclassified Sample")
    plt.tight_layout()
    plt.show()
else:
    print("No misclassified samples found")

### Learning Curves

In [ ]:
from sklearn.model_selection import learning_curve
from imblearn.pipeline import Pipeline as ImbPipeline

print("Computing learning curves for knowledge-driven model")
print("NOTE: Learning curve uses ORIGINAL training data with undersampling in pipeline")
print(f"Original training set: {X_train[feature_list_C].shape[0]} samples")
print(f"Class distribution: {pd.Series(y_train_ravel).value_counts().to_dict()}\n")

# Create pipeline with undersampling + model
# This ensures undersampling is applied at each training size
pipeline = ImbPipeline([
    ('sampler', RandomUnderSampler(random_state=42)),
    ('model', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))
])

# Use ORIGINAL training data (X_train, y_train_ravel)
# Learning curve will apply undersampling internally for each fold
train_sizes, train_scores, val_scores = learning_curve(
    pipeline,
    X_train[feature_list_C],  # Use original training data with selected features
    y_train_ravel,            # Use original training labels
    cv=5,
    train_sizes=np.linspace(0.1, 1.0, 10),
    scoring='f1_macro',
    n_jobs=-1,
    random_state=42
)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, label='Training score', color='blue', marker='o')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.15, color='blue')
plt.plot(train_sizes, val_mean, label='Cross-validation score', color='red', marker='s')
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.15, color='red')
plt.xlabel('Training Set Size (from original 1043 samples)')
plt.ylabel('F1 Score (Macro)')
plt.title('Learning Curve - Knowledge-Driven Model (12 genes)\nWith Undersampling Pipeline')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Final training score: {train_mean[-1]:.3f}")
print(f"Final validation score: {val_mean[-1]:.3f}")
print(f"Gap: {train_mean[-1] - val_mean[-1]:.3f}")

### Calibration Curves

In [ ]:
from sklearn.calibration import calibration_curve

print("Computing calibration curves for all three models\n")

# Get predicted probabilities for GBM class (class index 1 in binary after conversion)
y_test_binary = (y_test.values.ravel() == 3).astype(int)

# LASSO model probabilities
y_proba_lasso = rand_forest_lasso.predict_proba(X_test[feature_list_A])[:, 1]
prob_true_lasso, prob_pred_lasso = calibration_curve(
    y_test_binary, y_proba_lasso, n_bins=10, strategy='uniform'
)

# RFE model probabilities
y_proba_rfe = rand_forest_rfe.predict_proba(X_test[feature_list_B])[:, 1]
prob_true_rfe, prob_pred_rfe = calibration_curve(
    y_test_binary, y_proba_rfe, n_bins=10, strategy='uniform'
)

# Knowledge-driven model probabilities
y_proba_lit = rand_forest_lit.predict_proba(X_test[feature_list_C])[:, 1]
prob_true_lit, prob_pred_lit = calibration_curve(
    y_test_binary, y_proba_lit, n_bins=10, strategy='uniform'
)

# Plot all three calibration curves
plt.figure(figsize=(12, 8))
plt.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Perfectly calibrated')
plt.plot(prob_pred_lasso, prob_true_lasso, marker='o', linewidth=2, 
         label='LASSO (460 genes)', markersize=8)
plt.plot(prob_pred_rfe, prob_true_rfe, marker='s', linewidth=2,
         label='RFE (250 genes)', markersize=8)
plt.plot(prob_pred_lit, prob_true_lit, marker='^', linewidth=2,
         label='Knowledge (12 genes)', markersize=8)

plt.xlabel('Mean Predicted Probability', fontsize=12)
plt.ylabel('Fraction of Positives (True GBM Rate)', fontsize=12)
plt.title('Calibration Curves - GBM Detection\nAll Models with Undersampling', fontsize=14)
plt.legend(loc='upper left', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### ROC Curves and AUC

In [ ]:
from sklearn.metrics import roc_curve, auc

print("Computing ROC curves for all three models")
print("NOTE: All models trained with undersampling, tested on imbalanced data\n")

y_test_binary = (y_test.values.ravel() == 3).astype(int)

# Get predicted probabilities (consistent feature selection)
y_proba_lasso = rand_forest_lasso.predict_proba(X_test[feature_list_A])[:, 1]
y_proba_rfe = rand_forest_rfe.predict_proba(X_test[feature_list_B])[:, 1]
y_proba_lit = rand_forest_lit.predict_proba(X_test[feature_list_C])[:, 1]

# Calculate ROC curves
fpr_lasso, tpr_lasso, _ = roc_curve(y_test_binary, y_proba_lasso)
fpr_rfe, tpr_rfe, _ = roc_curve(y_test_binary, y_proba_rfe)
fpr_lit, tpr_lit, _ = roc_curve(y_test_binary, y_proba_lit)

# Calculate AUC scores
auc_lasso = auc(fpr_lasso, tpr_lasso)
auc_rfe = auc(fpr_rfe, tpr_rfe)
auc_lit = auc(fpr_lit, tpr_lit)

# Plot ROC curves
plt.figure(figsize=(10, 8))
plt.plot(fpr_lasso, tpr_lasso, label=f'LASSO ({len(feature_list_A)} genes) AUC={auc_lasso:.3f}', linewidth=2.5)
plt.plot(fpr_rfe, tpr_rfe, label=f'RFE ({len(feature_list_B)} genes) AUC={auc_rfe:.3f}', linewidth=2.5)
plt.plot(fpr_lit, tpr_lit, label=f'Knowledge ({len(feature_list_C)} genes) AUC={auc_lit:.3f}', linewidth=2.5)
plt.plot([0, 1], [0, 1], 'k--', label='Random classifier (AUC=0.50)', linewidth=1.5, alpha=0.7)

plt.xlabel('False Positive Rate (1 - Specificity)', fontsize=12)
plt.ylabel('True Positive Rate (Sensitivity/Recall)', fontsize=12)
plt.title('ROC Curves - GBM Detection\nAll Models with Undersampling', fontsize=14)
plt.legend(loc='lower right', fontsize=10)
plt.grid(alpha=0.3)
plt.xlim([-0.02, 1.02])
plt.ylim([-0.02, 1.02])
plt.tight_layout()
plt.show()

### Precision-Recall Curves

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score

print("Computing Precision-Recall curves for all three models")
print("NOTE: PR curves are more informative than ROC for imbalanced datasets\n")

# Calculate PR curves (y_test_binary and probabilities already defined above)
precision_lasso, recall_lasso, _ = precision_recall_curve(y_test_binary, y_proba_lasso)
precision_rfe, recall_rfe, _ = precision_recall_curve(y_test_binary, y_proba_rfe)
precision_lit, recall_lit, _ = precision_recall_curve(y_test_binary, y_proba_lit)

# Calculate Average Precision scores
ap_lasso = average_precision_score(y_test_binary, y_proba_lasso)
ap_rfe = average_precision_score(y_test_binary, y_proba_rfe)
ap_lit = average_precision_score(y_test_binary, y_proba_lit)

# Baseline = class prevalence in test set
baseline = y_test_binary.sum() / len(y_test_binary)
num_gbm = y_test_binary.sum()
num_healthy = len(y_test_binary) - num_gbm

plt.figure(figsize=(10, 8))
plt.plot(recall_lasso, precision_lasso, label=f'LASSO ({len(feature_list_A)} genes) AP={ap_lasso:.3f}', linewidth=2.5)
plt.plot(recall_rfe, precision_rfe, label=f'RFE ({len(feature_list_B)} genes) AP={ap_rfe:.3f}', linewidth=2.5)
plt.plot(recall_lit, precision_lit, label=f'Knowledge ({len(feature_list_C)} genes) AP={ap_lit:.3f}', linewidth=2.5)
plt.axhline(y=baseline, color='k', linestyle='--', 
            label=f'No-skill baseline (AP={baseline:.3f})\n{num_gbm} GBM / {num_healthy} Healthy', 
            linewidth=1.5, alpha=0.7)

plt.xlabel('Recall (Sensitivity)', fontsize=12)
plt.ylabel('Precision (Positive Predictive Value)', fontsize=12)
plt.title('Precision-Recall Curves - GBM Detection\nAll Models with Undersampling', fontsize=14)
plt.legend(loc='upper right', fontsize=10)
plt.grid(alpha=0.3)
plt.xlim([-0.02, 1.02])
plt.ylim([-0.02, 1.02])
plt.tight_layout()
plt.show()

### Stratified K-Fold Validation
We evaluate each feature subset with 5-fold Stratified CV using an undersampled RandomForest pipeline to quantify variance in macro F1 and GBM recall.

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import make_scorer, f1_score, recall_score
from imblearn.pipeline import Pipeline

# Custom scorer to avoid pos_label issue with macro F1
def macro_f1_scorer(y_true, y_pred):
    return f1_score(y_true, y_pred, average='macro', zero_division=0)

def gbm_recall_scorer(y_true, y_pred):
    return recall_score(y_true, y_pred, pos_label=3, zero_division=0)

feature_sets = [
    ("LASSO (323 genes)", feature_list_A),
    ("RFE (250 genes)", feature_list_B),
    ("Knowledge (12 genes)", feature_list_C)
]

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_rows = []

for name, feature_list in feature_sets:
    X_subset = X[feature_list]
    pipeline = Pipeline([
        ('sampler', RandomUnderSampler(random_state=42)),
        ('model', RandomForestClassifier(
            n_estimators=100,
            random_state=42,
            n_jobs=-1
        ))
    ])
    scores = cross_validate(
        pipeline,
        X_subset,
        y.values.ravel(),
        cv=cv,
        scoring={
            'macro_f1': make_scorer(macro_f1_scorer),
            'gbm_recall': make_scorer(gbm_recall_scorer)
        },
        n_jobs=-1,
        return_train_score=False
    )
    cv_rows.append({
        'Feature Set': name,
        'Macro F1 (mean)': float(np.mean(scores['test_macro_f1'])),
        'Macro F1 (std)': float(np.std(scores['test_macro_f1'])),
        'GBM Recall (mean)': float(np.mean(scores['test_gbm_recall'])),
        'GBM Recall (std)': float(np.std(scores['test_gbm_recall']))
    })

cv_results_df = pd.DataFrame(cv_rows)
display(cv_results_df.round(3))

plt.figure(figsize=(10, 5))
x = np.arange(len(cv_results_df))
width = 0.35

plt.bar(x - width/2, cv_results_df['Macro F1 (mean)'], width, 
        label='Macro F1', color='#457B9D', yerr=cv_results_df['Macro F1 (std)'], capsize=5)
plt.bar(x + width/2, cv_results_df['GBM Recall (mean)'], width,
        label='GBM Recall', color='#E63946', yerr=cv_results_df['GBM Recall (std)'], capsize=5)

plt.ylabel('Score')
plt.title('Stratified 5-Fold Cross-Validation Results')
plt.xticks(x, cv_results_df['Feature Set'], rotation=20, ha='right')
plt.legend()
plt.ylim(0, 1.0)
plt.tight_layout()
plt.show()
